<a href="https://colab.research.google.com/github/yesrin/CodingTest/blob/main/3%EC%9D%BC%EC%B0%A8_LCEL_LangChain_%EB%B3%B5%ED%95%A9%EC%B2%B4%EC%9D%B8_%EB%AA%A8%EB%8D%B8_%EC%B6%9C%EB%A0%A5%ED%8C%8C%EC%84%9C_Document_Loaders_%EB%B0%B0%ED%8F%AC_%EC%99%84%EC%84%B1_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 전체 패키지 설치
!pip install langchain>=0.3.0 langchain-openai langchain-community langchain-text-splitters

# 추가 유용한 패키지들
!pip install faiss-cpu tiktoken python-dotenv

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 28.0 MB/s eta 0:00:00


In [ ]:
# 라이브러리 설치
!pip install -q langchain langchain-openai tiktoken

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

프롬프트 템플릿 적용
  - langchain_core의 prompts 모듈에서 ChatPromptTemplate 클래스를 사용하여 천문학 전문가로서의 답변 형식의 프롬프트 템플릿 생성

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('You are an expert in astronomy. Answer the question. \
<Question> : {input}')
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in astronomy. Answer the question. <Question> : {input}'), additional_kwargs={})])

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = 'gpt-4o-mini')


# LCEL(LangChain Expression Language)
chain = prompt | llm        # | : 서로 다른 구성 요소를 연결

chain.invoke({'input' : '지구의 윤년에 주기는?'})

AIMessage(content='지구의 윤년 주기는 4년입니다. 그러나 정확하게 말하자면, 윤년은 다음과 같은 규칙에 따라 결정됩니다:\n\n1. 일반적으로, 4로 나누어떨어지는 해는 윤년입니다.\n2. 다만, 100으로 나누어떨어지는 해는 윤년이 아닙니다.\n3. 그러나 400으로 나누어떨어지는 해는 다시 윤년이 됩니다.\n\n이 규칙 덕분에 지구의 태양 주위를 도는 주기와 달력의 차이를 조정할 수 있습니다. 따라서 1600년, 2000년 같은 해는 윤년이지만, 1700년, 1800년, 1900년 같은 해는 윤년이 아닙니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 168, 'prompt_tokens': 30, 'total_tokens': 198, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CLo0YeWFsxzAMHmfpw1mZeWaxH0cq', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--50345f41-6252-484a-a057-3c13eae59657-0', usage_metadata={'input_tokens': 30, 'output_tokens': 168, 'total_tokens': 198, 'input_token_details': {'a

In [ ]:
result = chain.invoke({'input' : '지구의 윤년에 주기는?'})
result.content

'지구의 윤년 주기는 4년입니다. 하지만 윤년의 규칙은 조금 더 복잡합니다. 기본적으로 매 4년마다 1년이 추가되지만, 100년마다 윤년이 아니고, 그 중 400년마다 다시 윤년이 됩니다. 즉, 1900년은 윤년이 아니지만 2000년은 윤년입니다. 이를 통해 평균적으로 1년에 약 365.2425일이 되도록 조정하고 있습니다.'

LCEL
  - 기본 LLM 체인
  - 프롬프트, LLM, 문자열 출력 파서(StrOutputParser)를 연결하여 체인을 만들 수 있다

In [ ]:

# 라이브러리 임포트
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 간단한 프롬프트 템플릿 생성
template = "{country}의 수도는 어디인가요?"
prompt_template = PromptTemplate.from_template(template)

# 프롬프트 테스트
print(prompt_template.format(country="대한민국"))
print(prompt_template.format(country="미국"))



대한민국의 수도는 어디인가요?
미국의 수도는 어디인가요?


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser
prompt = ChatPromptTemplate.from_template('You are an expert in astronomy. Answer the question. \
<Question> : {input}')

llm = ChatOpenAI(model = 'gpt-4o-mini')

output_parser = StrOutputParser()

# LCEL chaining

chain = prompt | llm | output_parser

# chain 호출
result = chain.invoke({'input' : '지구의 윤년에 주기는?'})


print(result)

지구의 윤년 주기는 4년입니다. 그러나 일정한 규칙이 있어 추가적인 조건이 있습니다. 윤년은 다음과 같은 조건을 만족할 때 발생합니다:

1. 기본적으로, 해가 4로 나누어 떨어지는 해는 윤년입니다.
2. 하지만 100으로 나누어 떨어지는 해는 윤년이 아닙니다.
3. 그러나 400으로 나누어 떨어지는 해는 윤년입니다.

예를 들어, 2000년은 윤년이고(400으로 나눠짐), 1900년은 윤년이 아닙니다(100으로 나눠짐). 2024년은 윤년입니다.


템플릿 내용 변경하여 테스트

In [ ]:
# ChatOpenAI 모델 초기화
model = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1
)

# 출력 파서 초기화
output_parser = StrOutputParser()

# 체인 생성: 프롬프트 -> 모델 -> 출력 파서
prompt = PromptTemplate.from_template('{topic}에 대해서 쉽게 설명해 주세요.')

chain = prompt | model | output_parser

# 입력 딕셔너리
input_data = {"topic": "인공지능 모델의 학습 원리"}

# 체인 실행
result = chain.invoke(input_data)
print("=== 결과 ===")
print(result)




=== 결과 ===
인공지능 모델의 학습 원리는 데이터를 입력으로 받아들이고, 이 데이터를 분석하여 패턴을 찾아내는 과정입니다. 이 과정은 크게 입력층, 은닉층, 출력층으로 구성된 인공신경망을 사용하여 이루어집니다.

먼저, 입력층에는 데이터가 입력되고, 은닉층에서는 입력된 데이터를 분석하여 패턴을 찾아냅니다. 이후, 출력층에서는 은닉층에서 찾아낸 패턴을 바탕으로 결과를 출력합니다. 이때, 모델은 입력된 데이터와 실제 결과 사이의 차이를 최소화하는 방향으로 학습을 진행합니다.

이러한 과정을 반복하면서 모델은 점차적으로 데이터의 패턴을 학습하고, 새로운 데이터가 입력되었을 때 정확한 결과를 예측할 수 있도록 학습됩니다. 이렇게 학습된 모델은 실제 데이터를 분석하거나 예측하는 등 다양한 작업에 활용될 수 있습니다.


In [ ]:
# 영어 회화 예제
template_eng = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""
prompt_eng = PromptTemplate.from_template(template_eng)
model_eng = ChatOpenAI(model_name="gpt-4-turbo")
chain_eng = prompt_eng | model_eng | output_parser

# 질문 실행
input_question = {"question":"저는 식당에 가서 한국음식을 주문하고 싶어요."}
result_eng = chain_eng.invoke(input_question)
print("------- 영어 회화 예제--------")
print(result_eng)


# 또 다른 질문
input_question2 = {"question":"미국에서 피자 주문"}
result_eng2 = chain_eng.invoke(input_question2)
print("------- 피자 주문 예제 --------")
print(result_eng2)


------- 영어 회화 예제--------
영어 회화:
- Hello, could I have a table for one, please?
- Yes, could I see the menu? Thank you.
- I'd like to order the bibimbap and a side of kimchi, please.
- Could I also have a glass of water?
- Thank you so much.

한글 해석:
- 안녕하세요, 혼자 이용할 테이블 하나 주시겠어요?
- 네, 메뉴판 좀 볼 수 있을까요? 감사합니다.
- 비빔밥 하나랑 김치 사이드로 주문할게요.
- 물 한 잔도 주시겠어요?
- 정말 감사합니다.
------- 피자 주문 예제 --------
영어 회화:
- Hello, I'd like to order a pizza for delivery, please.
- Sure, what size pizza would you like to order?
- I’ll go with a large pizza. Could I get half pepperoni and half vegetable toppings?
- Absolutely, would you like to add any sides or drinks?
- Let's add an order of garlic bread and two cans of Coke, please.
- No problem. Can I have your delivery address and phone number?
- It's 123 Maple Street, Apartment 4C, and my phone number is 555-1234.
- Great, that will be $24 in total. It should be there in about 30 to 45 minutes.
- Thanks, see you soon!

한글 해석:
- 안녕하세요, 배달로 피자 주문하고 싶습니다.
- 네, 어떤 사이즈로 

문맥만을 기반으로 답변을 생성하는 체인

In [ ]:
# 필요한 패키지 설치
!pip install -qU faiss-cpu tiktoken grandalf langchain langchain_openai openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 19.5 MB/s eta 0:00:00


In [ ]:
!pip install grandalf

In [ ]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

import os
# OpenAI API 키를 환경 변수에 설정 (본인 키로 변경 필요)
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")


# FAISS 벡터 저장소에 텍스트 임베딩 저장
vectorstore = FAISS.from_texts(
    ["Tom is an AI engineer who loves programming!"],
    embedding= OpenAIEmbeddings(),
)

# 벡터 검색기 생성 (retriever)
retriever = vectorstore.as_retriever()

# 프롬프트 템플릿 정의
template = """Answer the question based only on the following context:
{context}

Question: {question}"""

# 문자열 템플릿을 LangChain 프롬프트 객체로 변환
prompt = ChatPromptTemplate.from_template(template)

# 모델 생성
model = ChatOpenAI(model_name="gpt-4o-mini")


# passthrough를 사용해서 입력값을 출력
passthrough = RunnableLambda(lambda x: x)

# 랭체인 체인구성
chain = (
    {'context':retriever, "question":passthrough}
    | prompt
    | model
    | StrOutputParser()
)


# 체인 내부의 그래프 노드 출력
print("Nodes :")
print(chain.get_graph().nodes)

# 체인 그래프 내 엣지 출력
print("Edges :")
print(chain.get_graph().edges)

# ASCII아트 형태 출력
print("Graph :")
chain.get_graph().print_ascii()

Nodes :
{'1f9da576ff754812b86376b539445125': Node(id='1f9da576ff754812b86376b539445125', name='Parallel<context,question>Input', data=<class 'langchain_core.runnables.base.RunnableParallel<context,question>Input'>, metadata=None), '6423365759534d6084d69cafa23b2047': Node(id='6423365759534d6084d69cafa23b2047', name='Parallel<context,question>Output', data=<class 'langchain_core.utils.pydantic.RunnableParallel<context,question>Output'>, metadata=None), 'ca6e29a2e360444b9a1c4b22ee8a1982': Node(id='ca6e29a2e360444b9a1c4b22ee8a1982', name='VectorStoreRetriever', data=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7dc6e64a03b0>, search_kwargs={}), metadata=None), 'faa22c34791a464d8506523f8d67db39': Node(id='faa22c34791a464d8506523f8d67db39', name='Lambda', data=RunnableLambda(lambda x: x), metadata=None), '3007b8e6668f4250afbc76a6cf2a3097': Node(id='3007b8e6668f4250afbc76a6cf2a3097', name='ChatPromptTemplate', da

LangChain Expression Language에서 라우팅을 수행하는 방법

LCEL로 라우팅을 수동으로 구현한 구조

In [ ]:
!pip install -U -q langchain langchain-core langchain-community langchain-openai

In [ ]:
# 필요한 라이브러리 임포트
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

# 사용자 질문을 분류하는 프롬프트 템플릿
prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 '수학', '과학' 또는 '기타' 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification :"""

)

chain = (
    prompt
    | ChatOpenAI(model='gpt-4')
    | StrOutputParser()
)


# 질문을 입력하여 체인을 호출
response1 = chain.invoke({"question": "2+2 는 무엇인가요?"})
response2 = chain.invoke({"question": "작용 반작용의 법칙은 무엇인가요?"})
response3 = chain.invoke({"question": "Google은 어떤 회사인가요?"})

print(response1)
print(response2)
print(response3)


# 수학 관련 질문을 처리하는 체인
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "깨봉선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4")
)

# 과학 관련 질문을 처리하는 체인
science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4")
)

# 기타 관련 질문을 처리하는 체인
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4")
)

# 각 체인을 호출하여 질문을 처리
math_response = math_chain.invoke({"question": "2+2 는 무엇인가요?"})
science_response = science_chain.invoke({"question": "작용 반작용의 법칙은 무엇인가요?"})
general_response = general_chain.invoke({"question": "Google은 어떤 회사인가요?"})

print(math_response)
print(science_response)
print(general_response)


수학
과학
기타
content='깨봉선생님께서 말씀하시기를, 2+2는 4입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 70, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-CLp8dQSVte9F2hj5Re3fyQh99rJR8', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--9126af47-1f85-4d09-8acd-a7b91366ab5f-0' usage_metadata={'input_tokens': 70, 'output_tokens': 33, 'total_tokens': 103, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content='아이작 뉴턴 선생님께서 말씀하시기를, 작용 반작용의 법칙은 자연법칙 중 하나로, 임의의 물체 A가 물체 B에게 힘을 가하면, 물체 B 역시 그와 같은 크기의 힘을 반대 방향으로 물체 A에게 가한다는 것입니다. 이 법칙은 뉴턴의 제3법칙이며, 과학적으로 중요한 기본 법칙으로서 모든 물리 현상에 적용됩니다.' additional_kwargs={'

In [ ]:
# 전체 패키지 설치
!pip install langchain>=0.3.0 langchain-openai langchain-community langchain-text-splitters

# 추가 유용한 패키지들
!pip install faiss-cpu tiktoken

# Chroma 벡터스토어를 사용할 때 필요한 패키지
!pip install langchain_chroma

사용자의 질문을 자동으로 분류하고, 그에 맞는 전문가 스타일로 답변을 생성하는 LangChain 기반 OpenAI 응답 시스템

RunnableLambda로 동적 분기

In [ ]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")


classification_prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 `수학`, `과학`, 또는 `기타` 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
)

classification_chain = (
    classification_prompt
    | ChatOpenAI(model='gpt-4o', temperature=0)
    | StrOutputParser()
)


math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "깨봉선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model = 'gpt-4o', temperature=0)
    | StrOutputParser()
)


science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o", temperature=0)
    | StrOutputParser()
)


general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o", temperature=0)
    | StrOutputParser()
)


def router(input: dict):
  category = classification_chain.invoke({'question': input['question']}).strip()
  if category == '수학':
    return math_chain
  elif category == '과학':
    return science_chain
  else:
    return general_chain


router_chain = RunnableLambda(lambda input: router(input).invoke(input))

questions = [
    "2+2는 무엇인가요?",
    "작용 반작용 법칙은 무엇인가요?",
    "Google은 어떤 회사인가요?",
]

for q in questions:
    print(f"\n질문: {q}")
    answer = router_chain.invoke({"question": q})
    print(f"\n답변: {answer}")



질문: 2+2는 무엇인가요?

답변: 깨봉선생님께서 말씀하시기를 2+2는 4입니다.

질문: 작용 반작용 법칙은 무엇인가요?

답변: 아이작 뉴턴 선생님께서 말씀하시기를, 작용 반작용 법칙은 뉴턴의 제3법칙으로 알려져 있으며, 이는 모든 작용에는 크기가 같고 방향이 반대인 반작용이 있다는 원리입니다. 즉, 어떤 물체가 다른 물체에 힘을 가하면, 두 번째 물체도 첫 번째 물체에 동일한 크기의 반대 방향의 힘을 가한다는 것입니다. 이 법칙은 물리학에서 힘의 상호작용을 이해하는 데 중요한 역할을 합니다.

질문: Google은 어떤 회사인가요?

답변: Google은 주로 인터넷 검색 엔진으로 잘 알려진 미국의 다국적 기술 회사입니다. 또한 온라인 광고, 클라우드 컴퓨팅, 소프트웨어 및 하드웨어 분야에서도 활동하고 있습니다. Google은 알파벳(Alphabet Inc.)의 자회사입니다.


In [ ]:
import os

from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

In [ ]:
# 전체 패키지 설치
!pip install langchain>=0.3.0 langchain-openai langchain-community langchain-text-splitters

# 추가 유용한 패키지들
!pip install faiss-cpu tiktoken

# Chroma 벡터스토어를 사용할 때 필요한 패키지
!pip install langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 8.5 MB/s eta 

In [ ]:
# LangChain 관련 라이브러리 임포트
from langchain_openai import ChatOpenAI
from langchain_core.outputs import LLMResult
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from typing import Dict, List, Any
import time
from datetime import datetime

LangChain의 언어 모델(Model)
  - LLM과 Chat Model 클래스는 각각 다른 형태의 입력과 출력을 다루는 언어 모델을 나타낸다
  - LLM : 단일 요청에 대한 복잡한 출력을 생성
  - Chat Model : 사용자와의 상호작용을 통한 연속적인 대화 관리에 적합


LangChain의 LLM 모델 파라미터


In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI


params = {
    'temperature': 0.7,
    'max_tokens': 100,
}


kwargs = {
    'frequency_penalty': 0.5,
    'presence_penalty': 0.5,
    'stop': ['\n']
}

model = ChatOpenAI(model= 'gpt-3.5-turbo', **params, model_kwargs= kwargs)


question = '태양계에서 가장 큰 행성은 무엇이니?'

response = model.invoke(input=question)

response.content

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: Parameters {'frequency_penalty', 'stop', 'presence_penalty'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if (await self.run_code(code, result,  async_=asy)):


'가장 큰 행성은 목성입니다. 목성은 태양계에서 가장 크고 질량이 큰 행성으로 알려져 있습니다.체적을 기준으로 볼 때, 목성은 지구의 1,321배에 달하는 크기를 갖고 있습니다.'

In [ ]:
params = {
    'temperature': 0.7,
    'max_tokens': 15,
}

response = model.invoke(input=question, **params)

print(response.content)

가장 큰 행성은 목성입니다. 목


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '너는 인공지능 질문에 댑변을 할 수 있어.'),
        ('user', '{user_input}'),
    ]
)

model = ChatOpenAI(model='gpt-4o-mini', max_tokens=100)

message = prompt.format_messages(user_input = '인공지능이 가장 발전한 나라와 대표 기술은 뭐지?')

# binding 이전에 모델을 직접 호출하여 응답 생성 및 출력
before_answer = model.invoke(message)

# binding 이전 출력
print(before_answer)

# 모델 호출 시 추가인수 전달 -> bind()
chain = prompt | model.bind(max_tokens= 50)

# binding 이후 체인 호출하고 응답셍성 및 출력
after_answer = chain.invoke({'user_input':'인공지능이 가장 발전한 나라와 대표 기술은?'})

# binding 이후 출력
print(after_answer)

content='인공지능이 가장 발전한 나라는 일반적으로 미국, 중국, 그리고 유럽의 몇몇 국가들이 꼽힙니다. \n\n1. **미국**: 실리콘밸리와 같은 혁신적인 기술 중심지 덕분에 많은 AI 스타트업과 대기업들이 존재합니다. 구글, 마이크로소프트, IBM, 페이스북 등이 주요 기업으로, 자연어 처리(NLP), 머신러닝, 이미지 인식 등의' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 42, 'total_tokens': 142, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CLpzVySG6JoSGsxpZnv3y2AULfKdS', 'service_tier': 'default', 'finish_reason': 'length', 'logprobs': None} id='run--04ea61f3-2c3c-4808-9cc7-020fd0ec37c2-0' usage_metadata={'input_tokens': 42, 'output_tokens': 100, 'total_tokens': 142, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content='인공지능이 가장 발전한 나라는 주

출력파서
  - 모델의 출력을 처리하고, 그 결과를 원하는 형식으로 변환하는 역할을 한다
  - 종류
    1. csv Parser
    2. json Parser

In [ ]:
# CSV Parser
# -> CommaSeperatedListOutputParser

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template= 'List five {subject}.\n{format_instructions}',
    input_variables= ['subject'],
    partial_variables= {'format_instructions': format_instructions}
)

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)


chain = prompt | llm | output_parser

chain.invoke({'subject':'popular Koeran cusine'})

['Kimchi', 'Bibimbap', 'Bulgogi', 'Tteokbokki', 'Japchae']

In [ ]:
# JSON Parser 예제

from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

class CusineRecipe(BaseModel):
  name: str = Field(description= 'name of cusine')
  recipe: str = Field(description= 'recipe to cook the cusine')

output_parser = JsonOutputParser(pydantic_object=CusineRecipe)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "name of cusine", "title": "Name", "type": "string"}, "recipe": {"description": "recipe to cook the cusine", "title": "Recipe", "type": "string"}}, "required": ["name", "recipe"]}
```


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

class CusineRecipe(BaseModel):
  name: str = Field(description= 'name of cusine')
  recipe: str = Field(description= 'recipe to cook the cusine')

output_parser = JsonOutputParser(pydantic_object=CusineRecipe)
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template = 'Answer the user query.\n{format_instructions}\n{query}\n',
    input_variables= ['query'],
    partial_variables={'format_instructions':format_instructions}
)

print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "name of cusine", "title": "Name", "type": "string"}, "recipe": {"description": "recipe to cook the cusine", "title": "Recipe", "type": "string"}}, "required": ["name", "recipe"]}\n```'} template='Answer the user query.\n{format_instructions}\n{query}\n'


In [ ]:

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

class CusineRecipe(BaseModel):
  name: str = Field(description= 'name of cusine')
  recipe: str = Field(description= 'recipe to cook the cusine')

output_parser = JsonOutputParser(pydantic_object=CusineRecipe)
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template = 'Answer the user query.\n{format_instructions}\n{query}\n',
    input_variables= ['query'],
    partial_variables={'format_instructions':format_instructions}
)

model = ChatOpenAI(model= 'gpt-4o-mini', temperature=0)

chain = prompt | model | output_parser

chain.invoke({'query':'Let me know how to cook Kimchi'})

{'name': 'Kimchi',
 'recipe': '1. Prepare the vegetables: Cut napa cabbage into quarters and rinse thoroughly. 2. Make the brine: Dissolve salt in water and soak the cabbage for 2-6 hours. 3. Prepare the spice paste: Blend garlic, ginger, Korean red pepper flakes, fish sauce, and sugar. 4. Mix the paste with the drained cabbage and other vegetables (like radish and green onions). 5. Pack the mixture into a jar, pressing down to remove air bubbles. 6. Let it ferment at room temperature for 1-5 days, then refrigerate.'}

문서 로더
  - LangChain의 문서로더를 사용해서 다양한 형식의 데이터 파일로 부터 문서로 로드할 수 있다
  - Document 객체로 로드되고, page_content에는 내용이 들어가고, metadata에는 메타데이터를 포함한다
  - txt 형식의 문서 뿐만아니라 pdf, word, ppt, xlsx,csv, hwp등 거의 모든 형식의 문서를 기반으로 LLM을 구동할 수 있다


In [ ]:

!pip install -q langchain pypdf unstructured pdf2image docx2txt pdfminer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 46.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 43.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 6.0 MB/s eta 0:00:00


In [ ]:

!pip install -q langchain-community

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


URL Document Loader
  - 웹에 기록된 글도 텍스트 형식으로 가져온다
  - WebBaseLoader와 UnstructureURLLoader

In [ ]:
from langchain.document_loaders import WebBaseLoader


loader = WebBaseLoader('https://n.news.naver.com/mnews/article/001/0015576911')

data = loader.load()

print(data[0].page_content)




















엔씨 '신더시티' 해보니…엔비디아 협업 기술력 돋보여














본문 바로가기






NAVER

뉴스


엔터




스포츠




날씨




프리미엄










검색










언론사별


정치


경제


사회


생활/문화


IT/과학


세계


랭킹


신문보기


오피니언


TV


팩트체크


알고리즘 안내


정정보도 모음
















연합뉴스

연합뉴스


구독

연합뉴스 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.
보러가기


연합뉴스 언론사 구독 해지되었습니다.




엔씨 '신더시티' 해보니…엔비디아 협업 기술력 돋보여









김주환 기자








김주환 기자




김주환 기자

구독
구독중




구독자
0


응원수
0



더보기









입력
2025.08.20. 오후 8:00


수정
2025.08.20. 오후 8:00



기사원문
 










추천




쏠쏠정보
0




흥미진진
0




공감백배
0




분석탁월
0




후속강추
0


 



댓글





본문 요약봇



본문 요약봇도움말
자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.
닫기








텍스트 음성 변환 서비스 사용하기



성별
남성
여성


말하기 속도
느림
보통
빠름

이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.
본문듣기 시작

닫기


 

글자 크기 변경하기



가1단계
작게


가2단계
보통


가3단계
크게


가4단계
아주크게


가5단계
최대크게






SNS 보내기



인쇄하기








게임스컴 엔비디아 행사서 시연배재현 빅파이어게임즈 대표 "의미 있는 K-게임 성과 내겠다"



'신더시티' 메인 화면[촬영 김주환]

PDF Document Loader

In [ ]:

from langchain.document_loaders import PyPDFLoader


loader = PyPDFLoader('/content/drive/MyDrive/AI_ChatBot/LLM_원본/자료/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf')


pages = loader.load_and_split()

In [ ]:

print(len(pages))

18


In [ ]:

pages

[Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hancom PDF 1.3.0.538', 'creationdate': '2022-07-29T09:03:16+09:00', 'author': 'kmd kdy', 'moddate': '2022-07-29T09:03:16+09:00', 'pdfversion': '1.4', 'source': '/content/drive/MyDrive/AI_ChatBot/LLM_원본/자료/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='혁신성장 정책금융 동향 : ICT 산업을 중심으로\n  CIS이슈리포트 2022-2호 | 1 |\n<요  약>▶혁신성장 정책금융기관*은 혁신성장산업 영위기업을 발굴·지원하기 위한 정책금융 가이드라인**에 따라 혁신성장 기술분야에 대한 금융지원을 강화하고 있음     * 산업은행, 기업은행, 수출입은행, 신용보증기금, 기술보증기금, 중소벤처기업진흥공단, 무역보험공사 등 11개 기관    ** 혁신성장 정책금융 지원 대상을 판단하는 기준으로, ‘9대 테마 – 46개 분야 – 296개 품목’으로 구성￮정책금융기관의 혁신성장 정책금융 공급규모는 2017년 24.1조 원에서 2021년 85.4조 원으로 크게 증가하여 국내 산업 구조의 미래 산업으로의 전환을 충실히 지원하고 있음￮본 보고서는 ICT 산업의 정책금융 지원 트렌드를 파악하고, 혁신성장 정책금융이 집중되는 주요 품목의 기술·시장 동향을 분석함▶혁신성장 ICT 산업은 정보통신(6개 분야, 47개 품목), 전기전자(5개 분야, 27개 품목), 센서측정(3개 분야, 19개 품목) 테마로 구성되며, 혁신성장 정책금융기관의 공급액 규모는 2021년 말 기준 16.9조 원으로 2017년 이후 연평균 39.2% 

In [ ]:
print(pages[0])

page_content='혁신성장 정책금융 동향 : ICT 산업을 중심으로
  CIS이슈리포트 2022-2호 | 1 |
<요  약>▶혁신성장 정책금융기관*은 혁신성장산업 영위기업을 발굴·지원하기 위한 정책금융 가이드라인**에 따라 혁신성장 기술분야에 대한 금융지원을 강화하고 있음     * 산업은행, 기업은행, 수출입은행, 신용보증기금, 기술보증기금, 중소벤처기업진흥공단, 무역보험공사 등 11개 기관    ** 혁신성장 정책금융 지원 대상을 판단하는 기준으로, ‘9대 테마 – 46개 분야 – 296개 품목’으로 구성￮정책금융기관의 혁신성장 정책금융 공급규모는 2017년 24.1조 원에서 2021년 85.4조 원으로 크게 증가하여 국내 산업 구조의 미래 산업으로의 전환을 충실히 지원하고 있음￮본 보고서는 ICT 산업의 정책금융 지원 트렌드를 파악하고, 혁신성장 정책금융이 집중되는 주요 품목의 기술·시장 동향을 분석함▶혁신성장 ICT 산업은 정보통신(6개 분야, 47개 품목), 전기전자(5개 분야, 27개 품목), 센서측정(3개 분야, 19개 품목) 테마로 구성되며, 혁신성장 정책금융기관의 공급액 규모는 2021년 말 기준 16.9조 원으로 2017년 이후 연평균 39.2% 지속 증가하고 있음￮ICT 산업의 공급액 규모 비중은 혁신성장 정책금융 총 공급 규모의 약 20% 수준임      * (‘17)18.7% → (’18)20.7%  → (’19)18.5%  → (’20)20.3%  → (’21)19.8%
혁신성장 정책금융 동향 :ICT 산업을 중심으로
오동찬 선임조사역 (dcoh@kcredit.or.kr)' metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hancom PDF 1.3.0.538', 'creationdate': '2022-07-29T09:03:16+09:00', 'author': 'kmd kdy', 'moddate': '2022-07-29T09:03:16+09:00', 'pdfversion': 

In [ ]:

print(pages[0].page_content)

혁신성장 정책금융 동향 : ICT 산업을 중심으로
  CIS이슈리포트 2022-2호 | 1 |
<요  약>▶혁신성장 정책금융기관*은 혁신성장산업 영위기업을 발굴·지원하기 위한 정책금융 가이드라인**에 따라 혁신성장 기술분야에 대한 금융지원을 강화하고 있음     * 산업은행, 기업은행, 수출입은행, 신용보증기금, 기술보증기금, 중소벤처기업진흥공단, 무역보험공사 등 11개 기관    ** 혁신성장 정책금융 지원 대상을 판단하는 기준으로, ‘9대 테마 – 46개 분야 – 296개 품목’으로 구성￮정책금융기관의 혁신성장 정책금융 공급규모는 2017년 24.1조 원에서 2021년 85.4조 원으로 크게 증가하여 국내 산업 구조의 미래 산업으로의 전환을 충실히 지원하고 있음￮본 보고서는 ICT 산업의 정책금융 지원 트렌드를 파악하고, 혁신성장 정책금융이 집중되는 주요 품목의 기술·시장 동향을 분석함▶혁신성장 ICT 산업은 정보통신(6개 분야, 47개 품목), 전기전자(5개 분야, 27개 품목), 센서측정(3개 분야, 19개 품목) 테마로 구성되며, 혁신성장 정책금융기관의 공급액 규모는 2021년 말 기준 16.9조 원으로 2017년 이후 연평균 39.2% 지속 증가하고 있음￮ICT 산업의 공급액 규모 비중은 혁신성장 정책금융 총 공급 규모의 약 20% 수준임      * (‘17)18.7% → (’18)20.7%  → (’19)18.5%  → (’20)20.3%  → (’21)19.8%
혁신성장 정책금융 동향 :ICT 산업을 중심으로
오동찬 선임조사역 (dcoh@kcredit.or.kr)


Word Document Loader

In [ ]:

from langchain.document_loaders import Docx2txtLoader


loader = Docx2txtLoader('/content/drive/MyDrive/AI_Agent/Agent_원본/Agent_data/해커톤.docx')

data = loader.load()

In [ ]:

data

[Document(metadata={'source': '/content/drive/MyDrive/AI_Agent/Agent_원본/Agent_data/해커톤.docx'}, page_content='<식단 기반 영양제 추천 서비스>\n\n\n\n아이디어 \n\n 일일 섭취량 기반 필요 영양성분 / 초과 영양성분 알림 서비스\n\n \n\n기획 배경 \n\n의학이 발달함에 따라 기대 수명 증가\n\n\n\n단순히 오래 사는 것이 아닌 건강하게 오래사는 법에 대한 관심 증가\n\n\n\n실질적으로 모든 필요 영양성분을 식단을 통해 섭취 불가\n\n\n\n가지고 있는 질병 및 건강 증진을 목적으로 사람들의 영양제 섭취량 증가\n\n\n\n필요 영양제와 복용법에 관한 관심 증가, 오남용으로 인한 부작용에 대한 걱정 / 너무 다양한 정보로 인한 혼란 야기\n\n필요 영양제와 복용법에 관한 관심 증가, 무분별한 정보로 인한 혼란 야기, 오남용 증가\n\n\n\n1인 가구 및 외식 비중의 증대로 섭취한 음식 정보에 대한 정확한 인지 불가능\n\n \n\n\n\n서비스 구성\n\n사전에 사용자 정보 등록\n\n성별, 나이, 키, 체중, 직업, 활동량, 야외활동량 (햇빛 노출) , 가지고 있는 질병, 기타 개선을 바라는 증상 등\n\n\n\n하루 섭취 정보 기입\n\n인터페이스 구성을 통한 직접 업로드 (ex : 김치찌개, 밥, 고등어 조림 등)  \n\n사진 업로드 : 기계 학습을 통한 이미지 학습으로 사진 업로드시 해당 음식과 자동적으로 매칭\n\n라이다 센서를 통한 인식 (라이다 센서를 통해 섭취하고자 하는 음식 업로드 / 활용 가능여부 체크 필요)\n\n\n\n영양 성분 부족 / 초과시 발생 가능한 증상 및 부작용 정보제공\n\n영양제일 경우 섭취량을 초과하지 않는 함량을 선별하여 정보 제공 혹은 검색 할 수 있도록 분류\n\n\n\n업로드한 섭취 정보를 기반 일일 권장량 대비 섭취 영양성분 확인\n\n\n\n사전에 입력한 사용자 정보를 바탕으로 추가로 필요

In [ ]:

data[0].metadata

{'source': '/content/drive/MyDrive/AI_Agent/Agent_원본/Agent_data/해커톤.docx'}

In [ ]:

data[0].page_content

'<식단 기반 영양제 추천 서비스>\n\n\n\n아이디어 \n\n 일일 섭취량 기반 필요 영양성분 / 초과 영양성분 알림 서비스\n\n \n\n기획 배경 \n\n의학이 발달함에 따라 기대 수명 증가\n\n\n\n단순히 오래 사는 것이 아닌 건강하게 오래사는 법에 대한 관심 증가\n\n\n\n실질적으로 모든 필요 영양성분을 식단을 통해 섭취 불가\n\n\n\n가지고 있는 질병 및 건강 증진을 목적으로 사람들의 영양제 섭취량 증가\n\n\n\n필요 영양제와 복용법에 관한 관심 증가, 오남용으로 인한 부작용에 대한 걱정 / 너무 다양한 정보로 인한 혼란 야기\n\n필요 영양제와 복용법에 관한 관심 증가, 무분별한 정보로 인한 혼란 야기, 오남용 증가\n\n\n\n1인 가구 및 외식 비중의 증대로 섭취한 음식 정보에 대한 정확한 인지 불가능\n\n \n\n\n\n서비스 구성\n\n사전에 사용자 정보 등록\n\n성별, 나이, 키, 체중, 직업, 활동량, 야외활동량 (햇빛 노출) , 가지고 있는 질병, 기타 개선을 바라는 증상 등\n\n\n\n하루 섭취 정보 기입\n\n인터페이스 구성을 통한 직접 업로드 (ex : 김치찌개, 밥, 고등어 조림 등)  \n\n사진 업로드 : 기계 학습을 통한 이미지 학습으로 사진 업로드시 해당 음식과 자동적으로 매칭\n\n라이다 센서를 통한 인식 (라이다 센서를 통해 섭취하고자 하는 음식 업로드 / 활용 가능여부 체크 필요)\n\n\n\n영양 성분 부족 / 초과시 발생 가능한 증상 및 부작용 정보제공\n\n영양제일 경우 섭취량을 초과하지 않는 함량을 선별하여 정보 제공 혹은 검색 할 수 있도록 분류\n\n\n\n업로드한 섭취 정보를 기반 일일 권장량 대비 섭취 영양성분 확인\n\n\n\n사전에 입력한 사용자 정보를 바탕으로 추가로 필요한 영양성분 알림 (해당 영양성분 섭취가 가능한 음식, 조리법 및 영양제 정보 제공 서비스 / 영양제일 경우 함량, 가격대별 정보제공)\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\

In [ ]:

print(data[0].page_content)

<식단 기반 영양제 추천 서비스>



아이디어 

 일일 섭취량 기반 필요 영양성분 / 초과 영양성분 알림 서비스

 

기획 배경 

의학이 발달함에 따라 기대 수명 증가



단순히 오래 사는 것이 아닌 건강하게 오래사는 법에 대한 관심 증가



실질적으로 모든 필요 영양성분을 식단을 통해 섭취 불가



가지고 있는 질병 및 건강 증진을 목적으로 사람들의 영양제 섭취량 증가



필요 영양제와 복용법에 관한 관심 증가, 오남용으로 인한 부작용에 대한 걱정 / 너무 다양한 정보로 인한 혼란 야기

필요 영양제와 복용법에 관한 관심 증가, 무분별한 정보로 인한 혼란 야기, 오남용 증가



1인 가구 및 외식 비중의 증대로 섭취한 음식 정보에 대한 정확한 인지 불가능

 



서비스 구성

사전에 사용자 정보 등록

성별, 나이, 키, 체중, 직업, 활동량, 야외활동량 (햇빛 노출) , 가지고 있는 질병, 기타 개선을 바라는 증상 등



하루 섭취 정보 기입

인터페이스 구성을 통한 직접 업로드 (ex : 김치찌개, 밥, 고등어 조림 등)  

사진 업로드 : 기계 학습을 통한 이미지 학습으로 사진 업로드시 해당 음식과 자동적으로 매칭

라이다 센서를 통한 인식 (라이다 센서를 통해 섭취하고자 하는 음식 업로드 / 활용 가능여부 체크 필요)



영양 성분 부족 / 초과시 발생 가능한 증상 및 부작용 정보제공

영양제일 경우 섭취량을 초과하지 않는 함량을 선별하여 정보 제공 혹은 검색 할 수 있도록 분류



업로드한 섭취 정보를 기반 일일 권장량 대비 섭취 영양성분 확인



사전에 입력한 사용자 정보를 바탕으로 추가로 필요한 영양성분 알림 (해당 영양성분 섭취가 가능한 음식, 조리법 및 영양제 정보 제공 서비스 / 영양제일 경우 함량, 가격대별 정보제공)





















기대효과 및 활용방안



전국민 건강한 식습관 형성 및 영양 섭취 장려

건강을 위한 건강한 식습관 재고 및 올바른 영양제 섭취에 관한 인식 및 사전 지식 

CSV Document Loader

In [ ]:

from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='/content/drive/MyDrive/AI_ChatBot/LLM_원본/자료/최종 필터링 데이터 (호텔, 지형) - 시트3.csv',
                   csv_args={
                       'delimiter':',',
                       'quotechar':'"',
                       'fieldnames':['Country','City','hotel','mountain','beach','plain']
                   })

data = loader.load()

In [ ]:

data[:6]

[Document(metadata={'source': '/content/drive/MyDrive/AI_ChatBot/LLM_원본/자료/최종 필터링 데이터 (호텔, 지형) - 시트3.csv', 'row': 0}, page_content='Country: \nCity: \nhotel: \nmountain: \nbeach: \nplain: '),
 Document(metadata={'source': '/content/drive/MyDrive/AI_ChatBot/LLM_원본/자료/최종 필터링 데이터 (호텔, 지형) - 시트3.csv', 'row': 1}, page_content='Country: Country\nCity: City\nhotel: hotel\nmountain: mountain\nbeach: beach\nplain: plain'),
 Document(metadata={'source': '/content/drive/MyDrive/AI_ChatBot/LLM_원본/자료/최종 필터링 데이터 (호텔, 지형) - 시트3.csv', 'row': 2}, page_content='Country: Italy\nCity: Agrigento\nhotel: 237\nmountain: 1\nbeach: 0\nplain: 0'),
 Document(metadata={'source': '/content/drive/MyDrive/AI_ChatBot/LLM_원본/자료/최종 필터링 데이터 (호텔, 지형) - 시트3.csv', 'row': 3}, page_content='Country: Italy\nCity: Alessandria\nhotel: 157\nmountain: 0\nbeach: 0\nplain: 1'),
 Document(metadata={'source': '/content/drive/MyDrive/AI

In [ ]:

print(data[0])

page_content='Country: 
City: 
hotel: 
mountain: 
beach: 
plain: ' metadata={'source': '/content/drive/MyDrive/AI_ChatBot/LLM_원본/자료/최종 필터링 데이터 (호텔, 지형) - 시트3.csv', 'row': 0}


In [ ]:

print(data[0].page_content)

Country: 
City: 
hotel: 
mountain: 
beach: 
plain: 


In [ ]:

for i, doc in enumerate(data[:5]):
  print(f'행 {i+1} : {doc.page_content}')

행 1 : Country: 
City: 
hotel: 
mountain: 
beach: 
plain: 
행 2 : Country: Country
City: City
hotel: hotel
mountain: mountain
beach: beach
plain: plain
행 3 : Country: Italy
City: Agrigento
hotel: 237
mountain: 1
beach: 0
plain: 0
행 4 : Country: Italy
City: Alessandria
hotel: 157
mountain: 0
beach: 0
plain: 1
행 5 : Country: Italy
City: Ancona
hotel: 113
mountain: 0
beach: 1
plain: 0


In [ ]:

full_txt = '\n'.join([doc.page_content for doc in data])

from langchain.schema import Document
full_doc = Document(page_content=full_txt, metadata={'source':'full_csv'})

print(full_doc.page_content[:500])

Country: 
City: 
hotel: 
mountain: 
beach: 
plain: 
Country: Country
City: City
hotel: hotel
mountain: mountain
beach: beach
plain: plain
Country: Italy
City: Agrigento
hotel: 237
mountain: 1
beach: 0
plain: 0
Country: Italy
City: Alessandria
hotel: 157
mountain: 0
beach: 0
plain: 1
Country: Italy
City: Ancona
hotel: 113
mountain: 0
beach: 1
plain: 0
Country: Italy
City: Aosta
hotel: 88
mountain: 1
beach: 0
plain: 0
Country: Italy
City: Arezzo
hotel: 185
mountain: 0
beach: 0
plain: 1
Country: It


LangChain 외 방식으로 텍스트 추출하기

In [ ]:

!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 65.3 MB/s eta 0:00:00


In [ ]:
import fitz
import requests
import os


def extract_text_from_pdf(pdf_path):
  document = fitz.open(pdf_path)

  text = ''
  for page_num in range(len(document)):
    page= document.load_page(page_num)
    text += page.get_text()
  return text

pdf_url = 'https://pdfobject.com/pdf/sample.pdf'
local_pdf_path = 'sample.pdf'

response = requests.get(pdf_url)

with open(local_pdf_path, 'wb') as f:
  f.write(response.content)


extract_text = extract_text_from_pdf(local_pdf_path)


print(extract_text)


os.remove(local_pdf_path)

Sample PDF
This is a simple PDF ﬁle. Fun fun fun.
Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Phasellus facilisis odio sed mi. 
Curabitur suscipit. Nullam vel nisi. Etiam semper ipsum ut lectus. Proin aliquam, erat eget 
pharetra commodo, eros mi condimentum quam, sed commodo justo quam ut velit. 
Integer a erat. Cras laoreet ligula cursus enim. Aenean scelerisque velit et tellus. 
Vestibulum dictum aliquet sem. Nulla facilisi. Vestibulum accumsan ante vitae elit. Nulla 
erat dolor, blandit in, rutrum quis, semper pulvinar, enim. Nullam varius congue risus. 
Vivamus sollicitudin, metus ut interdum eleifend, nisi tellus pellentesque elit, tristique 
accumsan eros quam et risus. Suspendisse libero odio, mattis sit amet, aliquet eget, 
hendrerit vel, nulla. Sed vitae augue. Aliquam erat volutpat. Aliquam feugiat vulputate nisl. 
Suspendisse quis nulla pretium ante pretium mollis. Proin velit ligula, sagittis at, egestas a, 
pulvinar quis, nisl.
Pellentesque sit amet lectus. 

In [ ]:

!pip install python-pptx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 11.2 MB/s eta 0:00:00


In [ ]:
from pptx import Presentation
from google.colab import files


def extract_text_from_pptx(pptx_path):
  presentation = Presentation(pptx_path)

  text = ''
  for slide in presentation.slides:  # 각 슬라이드 반복 처리
    for shape in slide.shapes:       # 슬라이드 내의 각 도형을 반복 처리
      if shape.has_text_frame:       # 도형안에 텍스트 프레임 여부 확인
        for paragraph in shape.text_frame.paragraphs:   # 텍스트프레임 내의 각 단락을 반복 처리
          text += paragraph.text + '\n'   # 각 단락의 텍스트를 추출하여 추가

  return text


uploaded = files.upload()

if uploaded:
    pptx_path = list(uploaded.keys())[0]
    extract_text = extract_text_from_pptx(pptx_path)
    print(extract_text)
else:
    print("PPTX 파일이 업로드되지 않았습니다.")

Saving Dickinson_Sample_Slides.pptx to Dickinson_Sample_Slides.pptx
Presentation Title
Author
Department
Date
Location
Presentation Title
Author
Department
Date
Location

Presentation Title
Author
Department
Date
Location
GraphTitle
Additional Notes - Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem
Chart Title
List Title
Lorem ipsum dolor sit amet 
Aenean commodo ligula eget dolor 
Cum sociis natoque penatibus et magnis dis parturient montes 
Donec quam felis, ultricies nec, pellentesque eu 
Lorem ipsum dolor sit amet, consectetuer adipiscing elit 
Aenean massa 
Aenean commodo ligula eget dolor
Slide Title
Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascet

In [ ]:

!pip install python-docx

In [ ]:
from docx import Document
from google.colab import files


def extract_text_from_docx(docx_path):
  document = Document(docx_path)

  text = ''
  for para in document.paragraphs:    # 문서의 각 단락 반복 처리
    text += para.text + '\n'          # 각 단락의 텍스트와 개행문자 추가
  return text


uploaded = files.upload()


if uploaded:
    docx_path = list(uploaded.keys())[0]
    extract_text = extract_text_from_docx(docx_path)
    print(extract_text)
else:
    print("DOCX 파일이 업로드되지 않았습니다.")

In [ ]:

!pip install openpyxl

In [ ]:
import openpyxl
from google.colab import files

def extract_text_from_xlsx(xlsx_path):
  workbook = openpyxl.load_workbook(xlsx_path)
  sheet = workbook.active     # 활성화된 시트 선택

  text = ''
  # 시트의 각 행을 반복 처리(values_only=True로 설정하여 셀 값만 추출)
  for row in sheet.iter_rows(values_only=True):
    text += ", ".join([str(cell) for cell in row]) + '\n'
  return text

uploaded = files.upload()


if uploaded:
    xlsx_path = list(uploaded.keys())[0]
    extract_text = extract_text_from_xlsx(xlsx_path)
    print(extract_text)
else:
    print("XLSX 파일이 업로드되지 않았습니다.")

Saving file_example_XLSX_10.xlsx to file_example_XLSX_10.xlsx
0, First Name, Last Name, Gender, Country, Age, Date, Id
1, Dulce, Abril, Female, United States, 32, 15/10/2017, 1562
2, Mara, Hashimoto, Female, Great Britain, 25, 16/08/2016, 1582
3, Philip, Gent, Male, France, 36, 21/05/2015, 2587
4, Kathleen, Hanner, Female, United States, 25, 15/10/2017, 3549
5, Nereida, Magwood, Female, United States, 58, 16/08/2016, 2468
6, Gaston, Brumm, Male, United States, 24, 21/05/2015, 2554
7, Etta, Hurn, Female, Great Britain, 56, 15/10/2017, 3598
8, Earlean, Melgar, Female, United States, 27, 16/08/2016, 2456
9, Vincenza, Weiland, Female, United States, 40, 21/05/2015, 6548



In [ ]:
# olefile 라이브러리 설치 (OLE 구조 파일 처리를 위해 사용, HWP 파일에 필요)
!pip install olefile

In [ ]:
import olefile
import zlib
import struct
import re
from google.colab import files

def get_hwp_text(filename):
  f = olefile.OleFileIO(filename)
  dirs = f.listdir()    # 파일 내부 디렉토리 구조 가져오기

  # 유효한 hwp 파일인지 확인(필수 스트림 존재 여부 확인)
  if ['FileHeader'] not in dirs or ['\x05HwpSummaryInformation'] not in dirs:
    raise Exception('Not Valid HWP')

  header = f.openstream('FileHeader')
  header_data = header.read()
  is_compressed = (header_data[36] & 1)

  nums = []
  for d in dirs:
    if d[0] == 'BodyText':
      nums.append(int(d[1][len('Section'):]))


  sections = ['BodyText/Section' + str(x) for x in sorted(nums)]

  text = ''
  for section in sections:
    bodytext = f.openstream(section)
    data = bodytext.read()
    if is_compressed:
      unpacked_data = zlib.decompress(data, -15)
    else:
      unpacked_data = data

    section_text = ''
    i = 0
    size = len(unpacked_data)


    while i < size:
      if i + 4 > size:
          break
      header = struct.unpack_from('<l', unpacked_data, i)[0]
      rec_type = header & 0x3ff
      rec_len = (header >> 20) & 0xfff
      if rec_type in [67]:
        if i + 4 + rec_len > size:
            break
        rec_data = unpacked_data[i + 4 : i + 4 + rec_len]
        try:
            section_text += rec_data.decode('utf-16')
        except UnicodeDecodeError:
            pass

      i += 4 + rec_len

    text += section_text
    text += '\n'
  return text


uploaded = files.upload()


if uploaded:
    hwp_path = list(uploaded.keys())[0]
    text = get_hwp_text(hwp_path)

    cleaned_text = re.sub(r'[^\x20-\x7E\uAC00-\uD7AF\n\r]', '', text)


    print(cleaned_text)
else:
    print("HWP 파일이 업로드되지 않았습니다.")